In [16]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [4]:
# Plot candle price
pair ="EUR_USD"
granularity = "H1"

In [5]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [8]:
df.describe() #show for numerical values

,volume
count,5000.000000
mean,3969.037600
std,2928.376527
min,125.000000
25%,2091.500000
50%,3270.500000
75%,4989.250000
max,27524.000000


In [9]:
non_cols = ['time', 'volume']

In [10]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [13]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [17]:
df_plot = df.iloc[-100:]

In [21]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, 
    open = df_plot.mid_o,
    high = df_plot.mid_h,
    low = df_plot.mid_l,
    close = df_plot.mid_c
))
fig.show()